### Inhoudsopgave
- [Basis](./Document.ipynb#basis)
- [Aanvullend](./Document.ipynb#aanvullend)
  - [Algemeen](./Document.ipynb#algemeen)
- [OOP](./Document.ipynb#oop)
- [C++](./Document.ipynb#cpp)
- [Uitbreiding](./Document.ipynb#uitbreiding)


<a id="basis"></a>
#### Basis
##### 1. useful and correct class (explain why)
   
- `Cell` stelt één Sudoku-vakje voor. Deze klasse bewaart alleen de waardes en de status van de cell.
- ik bewaar de waardes via `int m_value;`, lees de waarde met `int  value() const;` en als laate wijzig ik de waarde van de cell door deze lijn `void setValue(int v);`
- `bool m_fixed;`
    - True = fixed cells  
    - False = Speler kan cell invullen.
    - isFixed = leest de cell of fixed is of niet
    - setFixed = ga ik zorgen dat cell fixed moet zijn of niet
```cpp
namespace sudoku {

class Cell
{
public:
    Cell();

    int  value() const;
    void setValue(int v);

    bool isFixed() const;
    void setFixed(bool fixed);

private:
    int m_value;
    bool m_fixed; 
};

} // namespace sudoku
```
##### 2. useful and correct abstraction (explain why)
- Ik gebruik abstraction door abstracte basisclass `AbstractGame` te definieren.
- In deze class is er welke function de game moet hebben.
- De ui werkt met abstracte interface alleen en kent de implementatie van game niet.
```cpp
class AbstractGame
{
public:
    virtual ~AbstractGame() = default;
    virtual void reset() = 0;
    virtual void newGame(Difficulty difficulty) = 0;
    virtual Board& board() = 0;
    // ...
};
```
##### 3. useful and correct encapsulation (explain why)
- Ik gebruik encapsulation door data van mijn class in private te zetten. Het wordt alleen toegangelijk via public.
```cpp
    private:
    int m_value;
    bool m_fixed;
```
##### 4. useful and correct inheritance (explain why)
- Ik gebruik inheritance op bv bij spel logica:
1. Spel-logica: `Game` erft van de abstracte basisclass `AbstractGame`.
```cpp
class Game : public AbstractGame
{
public:
    explicit Game(Difficulty difficulty = Difficulty::Easy);
    void reset() override;
    // ...
};
```

##### 5. useful and correct polymorphism (explain why)

- Ik gebruik polymorfism doordat de UI werkt met de abstracte basisklasse `AbstractGame`, terwijl er in werkelijkheid een `Game`-object achter zit. Via de `virtual` functies in `AbstractGame` worden de overrides in `Game` uitgevoerd.

```cpp
private:
    sudoku::AbstractGame& m_game; 
```

##### 6. useful and correct object composition (explain why)
- Ik gebruik composition doordat `Board` een 2D-array van `Cell` bevatten.
```cpp
private:
    Cell m_cells[Size][Size];
```
##### 7. useful and correct base class
- Ik gebruik `AbstractGame` als base class voor de spel-logica. Deze class definieert welke functies elke game implementatie moet hebben.
```cpp
class AbstractGame
{
public:
    virtual ~AbstractGame() = default;
    virtual void reset() = 0;
    virtual void newGame(Difficulty difficulty) = 0;
    virtual Board& board() = 0;
    // ...
};
```
##### 8. useful and correct abstract base class
- `AbstractGame` is een abstract base class en het heeft meerdere pure virtuele functions.
```cpp
class AbstractGame
{
public:
    virtual ~AbstractGame() = default;
    virtual void reset() = 0;
    virtual void newGame(Difficulty difficulty) = 0;
    virtual Board& board() = 0;
    // ...
};
```
##### 9.  useful and correct virtual function
- `AbstractGame` dat ik heb geimplementeerd heeft een virtuele functions.
```cpp
virtual ~AbstractGame() = default;
```
##### 10.  no mistake in object-oriented programming
- In mijn huidige ontwerp volg ik de basisprincipes van OOP.
- Tot nu toe werkt de applicatie stabiel.

<a id="aanvullend"></a>
### Aanvullend
<a id="algemeen"></a>
#### Algemeen
1. [x] clean main (i.e. nothing in the main that should be in a class)
2. [x] no globals, but statics if needed
3. [x] correct protections
4. [x] maintainability by clean uniform code style and good function naming and/or comments everywhere
5. [x] separate header files
6. [x] one complete project that compiles and does not crash
7. [x] fully working project
8. [x] sufficient git commits (+/- weekly)
9. [x] correct files on git
10. [x] working build manual as readme on GitHub (project must be possible to build from scratch on a clean PC)

<a id="oop"></a>
#### OOP
1. [x] at least 2 default constructors
- `Cell::Cell()` stuurt nu door naar de parameterized versie `Cell(0, false)` zodat alle initialisatie op één plek zit.
- `Board::Board()` stuurt door naar `Board(0)` en hergebruikt de invullus voor elke cel.
```cpp
Cell::Cell() : Cell(0, false) {}
Board::Board() : Board(0) {}
```
2. [x] at least 2 parameterized constructors
- `Cell::Cell(int value, bool fixed)` geeft meteen een waarde en fixed-status.
- `Board::Board(int fillValue)` vult alle cellen met dezelfde startwaarde.
```cpp
Cell::Cell(int value, bool fixed) : m_value(value), m_fixed(fixed) {}
Board::Board(int fillValue)
{
    for (int r = 0; r < Size; ++r) {
        for (int c = 0; c < Size; ++c) {
            m_cells[r][c].setValue(fillValue);
            m_cells[r][c].setFixed(false);
        }
    }
}
```
3. [x] at least 2 copy constructors
- `Cell::Cell(const Cell&)` kopieert waarde en fixed-status.
- `Board::Board(const Board&)` kopieert elk vakje van het andere bord.
```cpp
Cell::Cell(const Cell& other)
    : m_value(other.m_value)
    , m_fixed(other.m_fixed)
{}
Board::Board(const Board& other)
{
    for (int r = 0; r < Size; ++r) {
        for (int c = 0; c < Size; ++c) {
            m_cells[r][c].setValue(other.m_cells[r][c].value());
            m_cells[r][c].setFixed(other.m_cells[r][c].isFixed());
        }
    }
}
```
4. [x] at least 2 destructors
- `Board::~Board()` roept `clear()` zodat er geen oude waardes blijven hangen.
- `Game::~Game()` wist het bord en zet `m_solution` terug leeg.
```cpp
Board::~Board()
{
    clear();
}
Game::~Game()
{
    m_board.clear();
    m_solution = {};
}
```
5. [x] member initialization in constructors (the stuff behind a colon)
- `Game::Game` gebruikt een initializer list voor bord, difficulty en solution.
```cpp
Game::Game(Difficulty difficulty)
    : m_board()
    , m_difficulty(difficulty)
    , m_solution{}
{
    setupInitialBoard();
}
```
6. [x] constructor forwarding
- Ik laat de default constructors doorsturen naar bestaande overloads zodat alle initialisatie centraal blijft en ik geen dubbele code schrijf.
```cpp
Cell::Cell() : Cell(0, false) {}
Board::Board() : Board(0) {}
```
7. [x] useful proven (dynamic) polymorphism
- `Game` override alle pure virtuals van `AbstractGame`.
```cpp
class Game : public AbstractGame
{
public:
    void reset() override;
    void newGame(Difficulty difficulty) override;
};
```
8. [x] useful usage of "this" (if the code does not work without it)
- `this` is nodig als parent voor Qt-dialogen.
```cpp
QMessageBox::information(this, "Sudoku", "De oplossing is ingevuld.");
```
9. [x] useful member function
- `Board::isValid()` controleert rijen, kolommen en blokken op conflicten.
```cpp
bool Board::isValid() const
{
    for (int r = 0; r < Size; ++r) {
        if (rowHasConflict(r)) {
            return false;
        }
    }
    // ... kolommen en blokken
    return true;
}
```
10. [x] default values in function definition
- `Game` heeft standaard difficulty `Easy` in de constructor.
```cpp
explicit Game(Difficulty difficulty = Difficulty::Easy);
```
11. [x] useful member variabel
- `m_solution` houdt de volledige oplossing bij.
```cpp
Grid m_solution;
```
12. [x] useful getters and setters for member variables
- `Cell::value()` en `Cell::setValue()` geven gecontroleerde toegang.
```cpp
int Cell::value() const { return m_value; }
void Cell::setValue(int v) { m_value = v; }
```
13. [x] correct usage of inline function
- `Board::isInside(int row, int col)` staat nu inline in `board.h` zodat de bounds-check zonder extra call overhead gebeurt.
```cpp
inline bool Board::isInside(int row, int col) const
{
    return row >= 0 && row < Size && col >= 0 && col < Size;
}
```
14. [x] useful template function or class
- Ik maak via `hasDuplicateValue` één template helper die rijen, kolommen en blokken controleert op duplicaten met een lambda.
```cpp
namespace {
    template<typename ValueProvider>
    bool hasDuplicateValue(ValueProvider&& provider, int count)
    {
        bool seen[10] = { false };
        for (int idx = 0; idx < count; ++idx) {
            int v = provider(idx);
            if (v == 0) continue;
            if (seen[v]) return true;
            seen[v] = true;
        }
        return false;
    }
}
```
15. [ ] useful friend function or class


<a id="cpp"></a>
#### C++
1. everything in one or more self-made namespace(s)
- [x] Alle interne projectklassen en helpers staan in de self-made namespace `sudoku`.
```cpp
    namespace sudoku {
    //....
    }
```
2. 2 useful unsigned chars or other better usage of memory efficient type
- [x] Interne opslag geoptimaliseerd: `Cell::m_value` gebruikt `std::uint8_t` en `Game::Grid` gebruikt `std::array<std::array<std::uint8_t,9>,9>` om geheugen voor celwaarden te besparen.
```cpp
// Cell intern opgeslagen als uint8_t (cell.h)
std::uint8_t m_value;
// Grid alias in game.h gebruikt uint8_t elementen
using Grid = std::array<std::array<std::uint8_t, 9>, 9>;
```
3. at least 4 useful const references for variables
4. at least 4 useful const references for functions
5. at least 4 useful bool
6. dynamic memory allocation (new)
7. dynamic memory removing (delete)
8. 2 useful (modern) call-by-references
9. useful string class usage
10. useful container class
11. useful usage of nullptr
12. useful usage of (modern) file-I/O
13. useful exception handling
14. useful usage of lambda function
15. useful usage of threads

<a id="uitbreiding"></a>
####  Uitbreiding
1. useful Qt class
2. useful usage of signals/slots
3. test-driven development (= written test plan or unit tests)
4. solve bug ticket (with pull request or commit message issue link and issue branch)
5. report a bug ticket on another project
6. usage of a GUI
7. usage of OpenGL or other 3D engine
8. useful usage of an external library (not Qt)
9. project that communicates (e.g. UART, BT) with hardware
10. a nice extra that you think that should deserve grading (stuff you put time in and is not rewarded by an item above)